[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Deposit and Withdraw

This notebook shows how to deposit a quantity of a specific token into a Mango Account, and then withdraw it.

This involves using Mango's root and node banks. Root banks keep track of tokens deposited across all Mango accounts, and node banks hold the actual tokens themselves. Much of this is handled for you by the `Account` object but it's possible to perform all the steps manually if you need to alter parts of the process.

In [ ]:
import decimal
import mango


# Use this function to display balances because it always fetches fresh data
def show_balances(context, wallet, group, sol_token):
    # Find the current SPL token balance
    token_account = mango.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
    print("Wrapped SOL in wallet:", token_account.value)

    # Find the SOL-specific details in the account
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    for slot in accounts[0].slots:
        if slot.base_instrument == sol_token:
            print("Wrapped SOL in account", slot.net_value)
            print()
            return


# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Signers are effectively an empty CombinableInstruction that only carries the keys for
# signing transactions
signers: mango.CombinableInstructions = mango.CombinableInstructions.from_wallet(wallet)

# Create a 'devnet' Context
with mango.ContextBuilder.build(cluster_name="devnet") as context:
    # Mango accounts are per-Group, so we need to load the Group first.
    group = mango.Group.load(context)

    # Find the wrapped SOL token
    sol_token = context.instrument_lookup.find_by_symbol_or_raise("SOL")

    print("Before:")
    show_balances(context, wallet, group, sol_token)

    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    print("Depositing 0.5 SOL:")

    deposit_value = mango.InstrumentValue(sol_token, decimal.Decimal("0.5"))
    deposit_signatures = account.deposit(context, wallet, deposit_value)
    print("Deposit transaction signatures:", deposit_signatures)

    print("Waiting for deposit transaction to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, deposit_signatures, commitment="processed"
        )

    print("After deposit:")
    show_balances(context, wallet, group, sol_token)

    print("Withdrawing 0.5 SOL:")

    withdrawal_value = mango.InstrumentValue(sol_token, decimal.Decimal("0.5"))
    withdrawal_signatures = account.withdraw(context, wallet, withdrawal_value, False)
    print("Withdrawal transaction signatures:", withdrawal_signatures)

    print("Waiting for withdraw transaction to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, withdrawal_signatures, commitment="processed"
        )

    print("After withdrawal:")
    show_balances(context, wallet, group, sol_token)

print("Example complete.")